In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model with FP16 (Half Precision)
model_name = "google/long-t5-local-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)

Some weights of LongT5ForConditionalGeneration were not initialized from the model checkpoint at google/long-t5-local-base and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
legal_text = """
A complaint was made by the appellant to the Railway Rates Tribunal against station, to station rates as being unreasonable on sugarcane imposed by the respondent Railway Co. by their rates circular No. 8 of 1953 with effect from October 1, 1953.
The Railway company in their answer pointed out that the rates imposed by the rates circular 8 of 1953 had long before the date of the complainant ceased to be in force and a new rate had come into operation from February 10, 1960.
Thereafter the complainant was allowed to amend the complaint.
The prayers were for a declaration that the rates charged were unreasonable, that a direction of refund of the excess collected or which may be collected after the date of the amendment of the complaint over the reasonable rates that may be fixed by the Tribunal and, lastly, for fixation of the rates.
The main contentions of the respondent were that the Tribunal had no jurisdiction to entertain the complaint as regards the reasonableness of rates prior to the institution of the complaint and that the Tribunal had no jurisdiction to grant refund.
The Indian Railways Act underwent several amendments and by the amendment in December 1957, section 41 was changed.
As a result of the change cl.
I of section 41 read thus: "Any com plaint that a railway administration (a) is contravening the provisions of section 28, or (b) is charging for the carriage of any commodity between two stations a rate which is unreasonable or is levying any other charge which is unreasonable shall hear and decide any such complaint in accordance with the provisions of this Chapter.
" 334 The Tribunal held that it had no jurisdiction to entertain or try the complaint as regards the rates and charges to prior to the institution of the complaint and it had no jurisdiction to grant any refund.
The appellants came up in appeal to the Supreme Court.
Held, that the words "is charging" in clause (b) and "is levying" in clause (c) of section 41 (1) of the Act must be construed to mean "is demanding a price at the present time for services to be rendered".
The Railway Rates Tribunal had no jurisdiction to entertain or try the complaint as regards the reasonableness or otherwise of rates and charges made prior to the institution of the complaint.
When the Tribunal had no jurisdiction to consider the reasonableness, or otherwise of any charges made prior to the institution of the complaint, it follows necessarily that it could have no occasion to order any refund.
For the question of refund could arise only after a decision that the charges made were more than what was reasonable.
Held, further, that neither expressly nor by necessary implication has the Railway Rates Tribunal been given any jurisdiction to make any order for refund.
, Southern Railway vs The Railway Rates Tribunal, A.I.R. 1955 (Madras) 676, referred to.
After a complaint is made the Tribunal shall hear and decide the complaint.
The complaint being that something is unreasonable all that the Tribunal has to decide is whether that thing is unreasonable or not.
A finding that it is unreasonable does not involve any consideration or decision of what would flow from the finding.
In otherwords, in making the complaint the complainant can ask only for a declaration that the rate or charge is unreasonable andit is only this declaratory relief which the Tribunal has been authorised to give.
There is no provision that the Tribunal can also give a consequential relief.
The only other thing which the Tribunal is authorised to do in connection with the complaint is to fix "such rate or charge as it considers reasonable.
" In the absence of any thing to indicate to the contrary it is reasonable to think that this fixation can only be prospective, that is, the Tribunal in making this order fixing the reasonable rate or, charge will mention a future date for this to come incoporation. 335
"""
def chunk_text(text, chunk_size=4096):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

text = "The railway company pointed out that rates imposed by the rates circular had ceased to be in force."

def preprocess_text(text, max_length=4096):  
    tokens = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=max_length)
    print("Tokens:", tokens)  # Debug: Check the tokens
    return tokens.input_ids

def summarize_text(text):
    input_tokens = preprocess_text(text).to("cpu")  # Keep input on CPU
    model.to(device)  # Move model to GPU only during inference
    with torch.no_grad():
        summary_ids = model.generate(
            input_tokens.to(device),
            max_length=512,  
            num_beams=5,  
            repetition_penalty=2.0,  
            length_penalty=1.0,  
            early_stopping=True
        )
    print("Generated IDs:", summary_ids)  # Debug: Check raw IDs
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print("Decoded Summary:", summary)  # Debug: Check decoded summary
    model.to("cpu")  # Free GPU memory after processing
    torch.cuda.empty_cache()  # Manually clear GPU memory
    return summary

summary_ids = model.generate(
    input_tokens.to(device),
    max_length=256,  # Reduce max length
    num_beams=5,
    repetition_penalty=2.0,
    length_penalty=1.0,
    early_stopping=True
)


# Test the summarization
summary = summarize_text(text)
print("Summary:", summary)

"""
chunks = chunk_text(legal_text)
summaries = [summarize_text(chunk) for chunk in chunks]
final_summary = " ".join(summaries)

print(final_summary)"""


Summary: ssss


'\nchunks = chunk_text(legal_text)\nsummaries = [summarize_text(chunk) for chunk in chunks]\nfinal_summary = " ".join(summaries)\n\nprint(final_summary)'

In [5]:
# Test input text
legal_text = """
A complaint was made by the appellant to the Railway Rates Tribunal against station, to station rates as being unreasonable on sugarcane imposed by the respondent Railway Co. by their rates circular No. 8 of 1953 with effect from October 1, 1953.
The Railway company in their answer pointed out that the rates imposed by the rates circular 8 of 1953 had long before the date of the complainant ceased to be in force and a new rate had come into operation from February 10, 1960.
Thereafter the complainant was allowed to amend the complaint.
The prayers were for a declaration that the rates charged were unreasonable, that a direction of refund of the excess collected or which may be collected after the date of the amendment of the complaint over the reasonable rates that may be fixed by the Tribunal and, lastly, for fixation of the rates.
"""

# Preprocess text and generate tokens
def preprocess_text(text, max_length=4096):  
    try:
        tokens = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=max_length)
        print("Tokenized Text:", tokenizer.decode(tokens.input_ids[0]))  # Debugging: check tokenization
        return tokens.input_ids
    except Exception as e:
        print(f"Error during tokenization: {e}")
        return None

# Summarize the text
def summarize_text(text):
    input_tokens = preprocess_text(text)  # Preprocess input text
    if input_tokens is None:
        return "Error during tokenization."
    
    input_tokens = input_tokens.to("cpu")  # Keep input on CPU
    model.to(device)  # Move model to GPU only during inference
    
    with torch.no_grad():
        summary_ids = model.generate(
            input_tokens.to(device),  # Pass input tokens to GPU
            max_length=256,  # Set max length for summary
            num_beams=5,  # Use beam search for better quality
            early_stopping=True
        )
    
    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    model.to("cpu")  # Free GPU memory after processing
    torch.cuda.empty_cache()  # Manually clear GPU memory
    return summary

# Chunking function
def chunk_text(text, chunk_size=4096):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Split the legal text into chunks
chunks = chunk_text(legal_text)
summaries = [summarize_text(chunk) for chunk in chunks]
final_summary = " ".join(summaries)

# Print final summary
print("Final Summary:", final_summary)


Tokenized Text: A complaint was made by the appellant to the Railway Rates Tribunal against station, to station rates as being unreasonable on sugarcane imposed by the respondent Railway Co. by their rates circular No. 8 of 1953 with effect from October 1, 1953. The Railway company in their answer pointed out that the rates imposed by the rates circular 8 of 1953 had long before the date of the complainant ceased to be in force and a new rate had come into operation from February 10, 1960. Thereafter the complainant was allowed to amend the complaint. The prayers were for a declaration that the rates charged were unreasonable, that a direction of refund of the excess collected or which may be collected after the date of the amendment of the complaint over the reasonable rates that may be fixed by the Tribunal and, lastly, for fixation of the rates.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><